In [ ]:
import os

# set the DB paths
os.environ["PYLAB_DB_LOCAL"] = r""
os.environ["PYLAB_DB_OUT"] = r""
import pylab_dk
import pylab_dk.pltconfig.color_preset as colors
from pylab_dk.constants import cm_to_inch
from pylab_dk.file_organizer import FileOrganizer
from pylab_dk.data_process import DataProcess
from pylab_dk.data_plot import DataPlot
from pylab_dk.measure_manager import MeasureManager

In [ ]:
project_name = "Date-Material" # random name used for debug
# if paths not correctly set in the environment variables, you can reset them here
#FileOrganizer.reload_paths(local_db_path=, out_db_path=)
folder = FileOrganizer(project_name)

In [ ]:
measurement = MeasureManager(project_name)
#measurement.get_visa_resources()   # list all VISA resources
measurement.load_meter("6221","GPIB0::12::INSTR")
measurement.load_meter("2182","GPIB0::7::INSTR")
#measurement.load_meter("sr830","GPIB0::8::INSTR","GPIB0::9::INSTR")
#measurement.load_mercury_ips("TCPIP0::10.97.24.237::7020::SOCKET")
#measurement.load_mercury_ipc("TCPIP0::10.101.28.24::7020::SOCKET")
#measurement.load_ITC503("GPIB0::23::INSTR","GPIB0::24::INSTR")
#measurement.load_rotator()


## Safety Check (repeat for every meter)

In [ ]:
meter_name = "6221"

In [ ]:
# check the get_status function
measurement.instrs[meter_name][0].setup()
measurement.instrs[meter_name][0].uni_output("1uA", type_str="curr")
measurement.instrs[meter_name][0].get_output_status()

In [ ]:
# check the switch-off function (should return the output value to 0)
measurement.instrs[meter_name][0].setup()
measurement.instrs[meter_name][0].uni_output("1uA", type_str="curr")
print("before switch off: ", measurement.instrs[meter_name][0].get_output_status()[0])
measurement.instrs[meter_name][0].output_switch("off")
print("after switch off: ", measurement.instrs[meter_name][0].get_output_status()[0])
measurement.instrs[meter_name][0].output_switch("on")
print("after switch off: ", measurement.instrs[meter_name][0].get_output_status()[0])

In [ ]:
# check the ramp function (from current and from 0)
measurement.instrs[meter_name][0].setup()
measurement.instrs[meter_name][0].ramp_output("curr", "10uA", interval="1uA",sleep=0.5, from_curr=True)

In [ ]:
# check source sweep apply method
measurement.source_sweep_apply("curr", "dc", "6221", max_value="10uA", step_value="1uA", compliance="1V",
                               sweepmode="0-max-0")

In [ ]:
# check the step threshold for every source meter (not implemented)
measurement.instrs[meter_name][0].setup()
measurement.instrs[meter_name][0].uni_output(1, type_str="volt")
measurement.instrs[meter_name][0].uni_output(3, type_str="volt")

## Normal Single Sweep/Vary

In [ ]:
measurement.instrs["6221"][0].setup()
measurement.instrs["2182"][0].setup()

In [ ]:
mea_dict = measurement.get_measure_dict(("I_source_sweep_dc", "V_sense"),
                                        1E-3, 5E-5, 1, 1, "0-max-0", "", 0, 0,
                                        wrapper_lst=[measurement.instrs["6221"][0],
                                                     measurement.instrs["2182"][0]],
                                        compliance_lst=["10mA"])

In [ ]:
assert len(mea_dict["swp_idx"]) <= 1, "Only one sweep is allowed"
swp_idx = mea_dict["swp_idx"][0] if len(mea_dict["swp_idx"]) > 0 else None
measurement.live_plot_init(1, 1, 1) # according to the actual needs
for i in mea_dict["gen_lst"]:
    measurement.record_update(mea_dict["file_path"], mea_dict["record_num"], i)
    measurement.live_plot_update(0, 0, 0, i[swp_idx], i[1])

## Rotate Probe

In [ ]:
rot_dev = measurement.instrs["rotator"]
rot_dev.print_info()

In [ ]:
rot_dev.set_spd(3)
rot_dev.ramp_angle(360, progress=True, wait=True)

## Constrained Multi-Sweep

In [ ]:
measurement.instrs["6221"][0].setup()
measurement.instrs["2182"][0].setup()

In [ ]:
# use manual sweep list to achieve constrained multi-sweep
# define a main sweep which is the independent variables
# use list instead of ndarray or generator
max_val = 1 # 1V
npts = 51 # 50 points single direction
swp_lst_main = [i * max_val/(npts - 1) for i in range(0, 51, 1)]
# define a sub sweep which is the dependent variables
def dependent_func(x):
    return 2-x
swp_lst_sub = [dependent_func(i) for i in swp_lst_main]

mea_dict = measurement.get_measure_dict(("V_source_sweep_dc","V_source_sweep_dc", "I_sense"),
                                        "-h",
                                        wrapper_lst=[measurement.instrs["2400"][0],
                                                     measurement.instrs["6430"][0],
                                                     measurement.instrs["2400"][0]],
                                        compliance_lst=["10mA","10mA"],
                                        sweep_tables=[swp_lst_main, swp_lst_sub])


In [ ]:
assert len(mea_dict["swp_idx"]) >= 1, "no sweep found"
swp_idxs = mea_dict["swp_idx"]
measurement.live_plot_init(1, 1, 1)
for i in mea_dict["gen_lst"]:
    measurement.record_update(mea_dict["file_path"], mea_dict["record_num"], i)
    measurement.live_plot_update(0, 0, 0, i[swp_idxs[0]], i[2])

## Mapping